In [26]:
from collections import OrderedDict
from typing import List, Tuple
import time
import flwr as fl
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from flwr.server.strategy import FedAvg
from torchvision.models._api import Weights
from torchvision.transforms import transforms
from typing import Optional, Dict
from flwr.common import Metrics, parameters_to_ndarrays, ndarrays_to_parameters, FitRes
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import CIFAR10

from tinysmpc import VirtualMachine, PrivateScalar
from tinysmpc.fixed_point import fixed_point, float_point

DEVICE = torch.device("cpu")  # Try "cuda" to train on GPU
print(f"Training on {DEVICE} using PyTorch {torch.__version__} and Flower {fl.__version__}")

Training on cpu using PyTorch 1.13.1+cpu and Flower 1.1.0


In [27]:
CLASSES = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
NUM_CLIENTS = 5  # 10
EPOCH = 2

In [28]:

BATCH_SIZE = 32


def load_datasets():
    # Download and transform CIFAR-10 (train and test)
    transform = transforms.Compose(
        [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]
    )
    trainset = CIFAR10("./dataset", train=True, download=False, transform=transform)
    testset = CIFAR10("./dataset", train=False, download=False, transform=transform)

    # Split training set into 10 partitions to simulate the individual dataset
    partition_size = len(trainset) // NUM_CLIENTS
    print(f"Train Size {len(trainset)}")
    print(f"Partition Size {partition_size}")
    print(f"Test Size {len(testset)}")
    lengths = [partition_size] * NUM_CLIENTS
    datasets = random_split(trainset, lengths, torch.Generator().manual_seed(42))

    # Split each partition into train/val and create DataLoader
    trainloaders = []
    valloaders = []
    for ds in datasets:
        len_val = len(ds) // 10  # 10 % validation set
        len_train = len(ds) - len_val
        lengths = [len_train, len_val]
        ds_train, ds_val = random_split(ds, lengths, torch.Generator().manual_seed(42))
        trainloaders.append(DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True))
        valloaders.append(DataLoader(ds_val, batch_size=BATCH_SIZE))
    testloader = DataLoader(testset, batch_size=BATCH_SIZE)
    return trainloaders, valloaders, testloader


trainloaders, valloaders, testloader = load_datasets()

Train Size 50000
Partition Size 10000
Test Size 10000


In [29]:
class Net(nn.Module):
    def __init__(self) -> None:
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [30]:
def train(net, trainloader, cid, epochs: int):
    """Train the network on the training set."""
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(net.parameters())
    net.train()
    times = []
    for _ in range(epochs):
        # measure start time
        start_epoch = time.time()
        # stat training
        for images, labels in trainloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            optimizer.zero_grad()
            loss = criterion(net(images), labels)
            loss.backward()
            optimizer.step()
        # measure end time
        end_epoch = time.time()
        elapsed = end_epoch - start_epoch
        times.append(elapsed)

    avg_time = sum(times) / epochs
    print(f"CLIENT :{int(cid) + 1} \nTIME ELAPSED {avg_time}")


def test(net, testloader):
    """Evaluate the network on the entire test set."""
    criterion = torch.nn.CrossEntropyLoss()
    correct = 0
    total = 0
    loss = 0.0
    net.eval()
    with torch.no_grad():
        for images, labels in testloader:
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    loss /= len(testloader.dataset)
    accuracy = correct / total
    return loss, accuracy

In [31]:
class FlowerClient(fl.client.NumPyClient):
    def __init__(self, net, trainloader, valloader, cid):
        self.net = net
        self.trainloader = trainloader
        self.valloader = valloader
        self.cid = cid

    def set_parameters(self, parameters: List[np.ndarray]):
        params_dict = zip(self.net.state_dict().keys(), parameters)
        state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
        self.net.load_state_dict(state_dict, strict=True)

    def get_parameters(self, config):
        return [val.cpu().numpy() for _, val in self.net.state_dict().items()]

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        train(self.net, self.trainloader, self.cid, EPOCH)
        return self.get_parameters(self.net), len(self.trainloader), {}

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        loss, accuracy = test(self.net, self.valloader)
        return float(loss), len(self.valloader), {"accuracy": float(accuracy)}

In [32]:
def client_fn(cid: str) -> FlowerClient:
    """Create a Flower client representing a single organization."""

    # Load model
    net = Net().to(DEVICE)

    # Load data (CIFAR-10)
    # Note: each client gets a different trainloader/valloader, so each client
    # will train and evaluate on their own unique data
    trainloader = trainloaders[int(cid)]
    valloader = valloaders[int(cid)]

    # Create a  single Flower client representing a single organization
    return FlowerClient(net, trainloader, valloader, cid)

In [33]:
Q = 2657003489534545107915232808830590043
fixedPoint = np.vectorize(fixed_point)
floatPoint = np.vectorize(float_point)


class FedAvgSmc(FedAvg):
    def aggregate_fit(
            self,
            rnd: int,
            results: List[Tuple[fl.server.client_proxy.ClientProxy, FitRes]],
            failures: List[BaseException],
    ) -> Optional[Weights]:
        if not results:
            return None

        start_epoch = time.time()

        clients = []
        fit_results = []

        for client, fit_result in results:
            clients.append(client)
            fit_results.append(fit_result)

        fit_res_ndarray_parameters = [parameters_to_ndarrays(result.parameters) for result in
                                      fit_results]  # list of clients, each with a list of layers(each layer representing weights)

        fl_nodes = [VirtualMachine(f"Client: {client.cid}") for client in clients]

        num_layers = len(fit_results[0].parameters.tensors)

        layers_weights = {}

        layers_weights_smc = {}

        for layer in range(num_layers):  #loop through number of layers
            layers_weights[f"layer_{layer}"] = []
            for weights in fit_res_ndarray_parameters:
                layers_weights[f"layer_{layer}"].append(fixedPoint(weights[layer]))

        for layer in range(num_layers):
            fl_node_values = [PrivateScalar(tensor, node) for tensor, node in
                              zip(layers_weights[f'layer_{layer}'], fl_nodes)]

            fl_exchanged_shares = []
            fl_exchanged_shares_list = []
            layers_weights_smc[f"layer_{layer}"] = []

            for value in fl_node_values:
                fl_exchanged_shares.append(value.share_tensor(fl_nodes, Q))

            for client_shares in fl_exchanged_shares:
                fl_exchanged_shares_list = [share.value for share in client_shares.shares]

            layers_weights_smc[f'layer_{layer}'] = floatPoint(fl_exchanged_shares_list)

        for i, client in enumerate(clients):
            client_weights = []
            for layer in layers_weights_smc.values():
                client_weights.append(np.array(layer[i]))
            fit_results[i].parameters = ndarrays_to_parameters(client_weights)

        results = tuple(zip(clients, fit_results))

        end_epoch = time.time()
        elapsed = end_epoch - start_epoch

        print(f"ADDITIVE SHARING TIME ELAPSED {elapsed}")

        return super().aggregate_fit(rnd, results, failures)

In [34]:
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    # Multiply accuracy of each client by number of examples used
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"accuracy": sum(accuracies) / sum(examples)}


# The `evaluate` function will be by Flower called after every round
def evaluate(
    server_round: int,
    parameters: fl.common.NDArrays,
    config: Dict[str, fl.common.Scalar],
) -> Optional[Tuple[float, Dict[str, fl.common.Scalar]]]:
    net = Net().to(DEVICE)
    valloader = valloaders[0]

    params_dict = zip(net.state_dict().keys(), parameters)
    state_dict = OrderedDict({k: torch.Tensor(v) for k, v in params_dict})
    net.load_state_dict(state_dict, strict=True)

    loss, accuracy = test(net, valloader)
    print(f"ROUND {server_round} SERVER SIDE ACCURACY {accuracy}")
    return loss, {"accuracy": accuracy}

In [37]:
# Create FedAvg strategy
strategy = FedAvg(
    fraction_fit=1.0,  # Sample 100% of available clients for training
    fraction_evaluate=0.5,  # Sample 50% of available clients for evaluation
    min_fit_clients=5,  # Never sample less than 5 clients for training
    min_evaluate_clients=5,  # Never sample less than 5 clients for evaluation
    min_available_clients=5,  # Wait until 5 clients are available
    evaluate_metrics_aggregation_fn=weighted_average,  # pass the metric aggregation function
    evaluate_fn=evaluate,
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=10),
    strategy=strategy,
)

INFO flower 2023-03-08 15:45:11,099 | app.py:140 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-03-08 15:45:16,370	INFO worker.py:1518 -- Started a local Ray instance.
INFO flower 2023-03-08 15:45:19,474 | app.py:174 | Flower VCE: Ray initialized with resources: {'CPU': 4.0, 'object_store_memory': 2214467174.0, 'memory': 4428934350.0, 'node:127.0.0.1': 1.0}
INFO flower 2023-03-08 15:45:19,475 | server.py:86 | Initializing global parameters
INFO flower 2023-03-08 15:45:19,477 | server.py:270 | Requesting initial parameters from one random client
INFO flower 2023-03-08 15:45:25,006 | server.py:274 | Received initial parameters from one random client
INFO flower 2023-03-08 15:45:25,007 | server.py:88 | Evaluating initial parameters
INFO flower 2023-03-08 15:45:25,355 | server.py:91 | initial parameters (loss, other metrics): 0.07367552161216735, {'accuracy': 0.116}
INFO flower 2023-03-08 15:45:25,358 | server.py:101 | FL starting
DEBUG flower 20

ROUND 0 SERVER SIDE ACCURACY 0.116
(launch_and_fit pid=9664) CLIENT :1 
(launch_and_fit pid=9664) TIME ELAPSED 8.62890350818634
(launch_and_fit pid=15368) CLIENT :3 
(launch_and_fit pid=15368) TIME ELAPSED 9.58100426197052
(launch_and_fit pid=13612) CLIENT :2 
(launch_and_fit pid=13612) TIME ELAPSED 9.52465546131134
(launch_and_fit pid=9316) CLIENT :5 
(launch_and_fit pid=9316) TIME ELAPSED 9.583497643470764


DEBUG flower 2023-03-08 15:46:01,041 | server.py:229 | fit_round 1 received 5 results and 0 failures
WARNING flower 2023-03-08 15:46:01,058 | fedavg.py:242 | No fit_metrics_aggregation_fn provided


(launch_and_fit pid=9664) CLIENT :4 
(launch_and_fit pid=9664) TIME ELAPSED 6.918647408485413


INFO flower 2023-03-08 15:46:01,431 | server.py:116 | fit progress: (1, 0.05744132351875305, {'accuracy': 0.414}, 36.07115330000033)
DEBUG flower 2023-03-08 15:46:01,431 | server.py:165 | evaluate_round 1: strategy sampled 5 clients (out of 5)


...

(launch_and_fit pid=9664) CLIENT :4 
(launch_and_fit pid=9664) TIME ELAPSED 4.811644792556763


INFO flower 2023-03-08 15:51:50,863 | server.py:116 | fit progress: (10, 0.03455497491359711, {'accuracy': 0.624}, 385.5036381)
DEBUG flower 2023-03-08 15:51:50,864 | server.py:165 | evaluate_round 10: strategy sampled 5 clients (out of 5)


ROUND 10 SERVER SIDE ACCURACY 0.624


DEBUG flower 2023-03-08 15:51:57,365 | server.py:179 | evaluate_round 10 received 5 results and 0 failures
INFO flower 2023-03-08 15:51:57,367 | server.py:144 | FL finished in 392.00707820000025
INFO flower 2023-03-08 15:51:57,368 | app.py:192 | app_fit: losses_distributed [(1, 0.05741369733810424), (2, 0.045469286775588987), (3, 0.04195360206365585), (4, 0.03944564276337624), (5, 0.03794898006916046), (6, 0.036901649141311645), (7, 0.036060101568698884), (8, 0.03563987969458103), (9, 0.03543092755675316), (10, 0.035200512886047364)]
INFO flower 2023-03-08 15:51:57,370 | app.py:193 | app_fit: metrics_distributed {'accuracy': [(1, 0.4082), (2, 0.4958), (3, 0.5298), (4, 0.561), (5, 0.5795999999999999), (6, 0.59), (7, 0.6056000000000001), (8, 0.6136000000000001), (9, 0.6188), (10, 0.6202)]}
INFO flower 2023-03-08 15:51:57,371 | app.py:194 | app_fit: losses_centralized [(0, 0.07367552161216735), (1, 0.05744132351875305), (2, 0.045177886247634885), (3, 0.0421369491815567), (4, 0.03908089220

History (loss, distributed):
	round 1: 0.05741369733810424
	round 2: 0.045469286775588987
	round 3: 0.04195360206365585
	round 4: 0.03944564276337624
	round 5: 0.03794898006916046
	round 6: 0.036901649141311645
	round 7: 0.036060101568698884
	round 8: 0.03563987969458103
	round 9: 0.03543092755675316
	round 10: 0.035200512886047364
History (loss, centralized):
	round 0: 0.07367552161216735
	round 1: 0.05744132351875305
	round 2: 0.045177886247634885
	round 3: 0.0421369491815567
	round 4: 0.03908089220523834
	round 5: 0.037522343516349796
	round 6: 0.03672231686115265
	round 7: 0.03556534498929977
	round 8: 0.034846886694431305
	round 9: 0.03504034513235092
	round 10: 0.03455497491359711
History (metrics, distributed):
{'accuracy': [(1, 0.4082), (2, 0.4958), (3, 0.5298), (4, 0.561), (5, 0.5795999999999999), (6, 0.59), (7, 0.6056000000000001), (8, 0.6136000000000001), (9, 0.6188), (10, 0.6202)]}History (metrics, centralized):
{'accuracy': [(0, 0.116), (1, 0.414), (2, 0.494), (3, 0.536), 

In [ ]:
# Create FedAvgSmc strategy
strategy = FedAvgSmc(
    fraction_fit=1.0,  # Sample 100% of available clients for training
    fraction_evaluate=0.5,  # Sample 50% of available clients for evaluation
    min_fit_clients=5,  # Never sample less than 5 clients for training
    min_evaluate_clients=5,  # Never sample less than 5 clients for evaluation
    min_available_clients=5,  # Wait until 5 clients are available
    evaluate_metrics_aggregation_fn=weighted_average,
    evaluate_fn=evaluate,
)

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=10),
    strategy=strategy,
)

INFO flower 2023-03-08 18:07:07,622 | app.py:140 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)
2023-03-08 18:07:13,649	INFO worker.py:1518 -- Started a local Ray instance.
INFO flower 2023-03-08 18:07:17,832 | app.py:174 | Flower VCE: Ray initialized with resources: {'node:127.0.0.1': 1.0, 'object_store_memory': 1923488563.0, 'CPU': 4.0, 'memory': 3846977127.0}
INFO flower 2023-03-08 18:07:17,833 | server.py:86 | Initializing global parameters
INFO flower 2023-03-08 18:07:17,834 | server.py:270 | Requesting initial parameters from one random client
INFO flower 2023-03-08 18:07:23,605 | server.py:274 | Received initial parameters from one random client
INFO flower 2023-03-08 18:07:23,606 | server.py:88 | Evaluating initial parameters
INFO flower 2023-03-08 18:07:23,973 | server.py:91 | initial parameters (loss, other metrics): 0.07366065049171448, {'accuracy': 0.1}
INFO flower 2023-03-08 18:07:23,973 | server.py:101 | FL starting
DEBUG flower 2023

ROUND 0 SERVER SIDE ACCURACY 0.1
(launch_and_fit pid=20428) CLIENT :2 
(launch_and_fit pid=20428) TIME ELAPSED 8.539761424064636
(launch_and_fit pid=8220) CLIENT :5 
(launch_and_fit pid=8220) TIME ELAPSED 8.534636497497559
(launch_and_fit pid=6768) CLIENT :3 
(launch_and_fit pid=6768) TIME ELAPSED 8.555080652236938
(launch_and_fit pid=17540) CLIENT :4 
(launch_and_fit pid=17540) TIME ELAPSED 8.613381743431091


DEBUG flower 2023-03-08 18:08:00,967 | server.py:229 | fit_round 1 received 5 results and 0 failures


(launch_and_fit pid=20428) CLIENT :1 
(launch_and_fit pid=20428) TIME ELAPSED 7.361392259597778


WARNING flower 2023-03-08 18:08:02,632 | fedavg.py:242 | No fit_metrics_aggregation_fn provided


...

DEBUG flower 2023-03-08 18:10:01,903 | server.py:229 | fit_round 4 received 5 results and 0 failures


(launch_and_fit pid=8220) CLIENT :2 
(launch_and_fit pid=8220) TIME ELAPSED 5.68003237247467
ADDITIVE SHARING TIME ELAPSED 3.0120198726654053


INFO flower 2023-03-08 18:10:05,303 | server.py:116 | fit progress: (4, nan, {'accuracy': 0.101}, 161.32972159999917)
DEBUG flower 2023-03-08 18:10:05,304 | server.py:165 | evaluate_round 4: strategy sampled 5 clients (out of 5)


ROUND 4 SERVER SIDE ACCURACY 0.101


DEBUG flower 2023-03-08 18:10:14,394 | server.py:179 | evaluate_round 4 received 5 results and 0 failures
DEBUG flower 2023-03-08 18:10:14,396 | server.py:215 | fit_round 5: strategy sampled 5 clients (out of 5)
